In [14]:
import requests
from minsearch import AppendableIndex

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [3]:
search_description = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [4]:
import chat_assistant

In [5]:
tools = chat_assistant.Tools()
tools.add_tool(search, search_description)

In [6]:
tools.get_tools()

[{'type': 'function',
  'name': 'search',
  'description': 'Search the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'Search query text to look up in the course FAQ.'}},
   'required': ['query'],
   'additionalProperties': False}}]

In [7]:
from openai import OpenAI
client = OpenAI()

In [8]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)


In [9]:
chat.run()

You: I want to add an entry to the FAQ database


You: How do I run docker? docker run


You: stop


Chat ended.


## Let's add a new tool

In [19]:
def add_entry(question, answer):
    doc = {
        'question': question,
        'text': answer,
        'section': 'user added',
        'course': 'data-engineering-zoomcamp'
    }
    index.append(doc)
    
add_entry_description = {
    "type": "function",
    "name": "add_entry",
    "description": "Add an entry to the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question to be added to the FAQ database",
            },
            "answer": {
                "type": "string",
                "description": "The answer to the question",
            }
        },
        "required": ["question", "answer"],
        "additionalProperties": False
    }
}

In [20]:
tools.add_tool(add_entry, add_entry_description)

In [16]:
tools.get_tools()

[{'type': 'function',
  'name': 'search',
  'description': 'Search the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'Search query text to look up in the course FAQ.'}},
   'required': ['query'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'add_entry',
  'description': 'Add an entry to the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'question': {'type': 'string',
     'description': 'The question to be added to the FAQ database'},
    'answer': {'type': 'string', 'description': 'The answer to the question'}},
   'required': ['question', 'answer'],
   'additionalProperties': False}}]

In [22]:
chat.run()

You: How do I do well for module 1?


You: add this back to FAQ


You: stop


Chat ended.


In [23]:
index.docs[-1]

{'question': 'How do I do well for module 1?',
 'text': "To do well in Module 1, here are some key tips:\n\n1. **Understanding Concepts**: Make sure you grasp the fundamental concepts of Docker and Terraform. Reviewing official documentation and tutorials can be very helpful.\n\n2. **Hands-On Practice**: Set up a Docker environment and experiment with the commands and configurations discussed in the module. Practice is crucial for mastering these tools.\n\n3. **Resolving Errors**: Be attentive to error messages. For instance, if you encounter a `ModuleNotFoundError` for `psycopg2`, ensure you have it installed correctly using either `pip install psycopg2-binary` or `conda`.\n\n4. **Use Resources**: Leverage resources like forums, online tutorials, and your peers for help.\n\n5. **Stay Organized**: Keep your assignments and notes organized. Regularly review what you have learned to reinforce your understanding.\n\n6. **Ask Questions**: If you're stuck, ask questions during your sessions

In [26]:
search("how do I do well for module 1?")

[{'question': 'How do I do well for module 1?',
  'text': "To do well in Module 1, here are some key tips:\n\n1. **Understanding Concepts**: Make sure you grasp the fundamental concepts of Docker and Terraform. Reviewing official documentation and tutorials can be very helpful.\n\n2. **Hands-On Practice**: Set up a Docker environment and experiment with the commands and configurations discussed in the module. Practice is crucial for mastering these tools.\n\n3. **Resolving Errors**: Be attentive to error messages. For instance, if you encounter a `ModuleNotFoundError` for `psycopg2`, ensure you have it installed correctly using either `pip install psycopg2-binary` or `conda`.\n\n4. **Use Resources**: Leverage resources like forums, online tutorials, and your peers for help.\n\n5. **Stay Organized**: Keep your assignments and notes organized. Regularly review what you have learned to reinforce your understanding.\n\n6. **Ask Questions**: If you're stuck, ask questions during your sessio

## Frameworks

Let's use PydanticAI

    pip install pydantic-ai

In [29]:
from pydantic_ai import Agent, RunContext
from typing import Dict

In [30]:
chat_agent = Agent(  
    'openai:gpt-4o-mini',
    system_prompt=developer_prompt
)

In [31]:
@chat_agent.tool
def search_tool(ctx: RunContext, query: str) -> Dict[str, str]:
    """
    Search the FAQ for relevant entries matching the query.

    Parameters
    ----------
    query : str
        The search query string provided by the user.

    Returns
    -------
    list
        A list of search results (up to 5), each containing relevance information 
        and associated output IDs.
    """
    print(f"search('{query}')")
    return search(query)

In [32]:
@chat_agent.tool
def add_entry_tool(ctx: RunContext, question: str, answer: str) -> None:
    """
    Add a new question-answer entry to FAQ.

    This function creates a document with the given question and answer, 
    tagging it as user-added content.

    Parameters
    ----------
    question : str
        The question text to be added to the index.

    answer : str
        The answer or explanation corresponding to the question.

    Returns
    -------
    None
    """
    return add_entry(question, answer)

In [35]:
user_prompt = "I just discovered the course. Can I join now?"

agent_run = await chat_agent.run(user_prompt)

search(Can I join the course now?)


In [36]:
agent_run.output

"Yes, you can still join the course even if you discover it after it has started. You are eligible to submit homework assignments, but keep in mind that there are deadlines for turning in the final projects, so it's best not to postpone everything to the last minute.\n\nWould you like more information about the course schedule or requirements?"